In [1]:
import pandas as pd
import numpy as np

In [3]:
df_base = pd.read_csv("../data/BR-Football-Dataset.csv")
df_2024 = pd.read_csv("../data/sport_serie_b_2024_manual.csv")


In [4]:
df_base["date"] = pd.to_datetime(df_base["date"], errors="coerce")
df_2024["date"] = pd.to_datetime(df_2024["date"], errors="coerce")


In [5]:
anos = [2019, 2022, 2023]
df_base = df_base[df_base["date"].dt.year.isin(anos)]


In [6]:
df_full = pd.concat([df_base, df_2024], ignore_index=True)


In [9]:
df_full["date"] = pd.to_datetime(df_full["date"], errors="coerce")
df_full["date"].dt.year.value_counts().sort_index()


date
2019    1069
2022    1089
2023    1088
2024      38
Name: count, dtype: int64

In [11]:
df_sb = df_full[df_full["tournament"].str.contains("Serie B", case=False, na=False)].copy()
df_sb["date"].dt.year.value_counts().sort_index(), df_sb.shape


(date
 2019    378
 2022    380
 2023    380
 2024     38
 Name: count, dtype: int64,
 (1176, 18))

In [12]:
mask_sport = (
    df_sb["home"].astype(str).str.contains("Sport Recife", case=False, na=False) |
    df_sb["away"].astype(str).str.contains("Sport Recife", case=False, na=False)
)

df_sport = df_sb.loc[mask_sport].copy()
df_sport["date"].dt.year.value_counts().sort_index(), df_sport.shape


(date
 2019    38
 2022    38
 2023    38
 2024    38
 Name: count, dtype: int64,
 (152, 18))

In [13]:
import numpy as np

df_sport["local"] = np.where(
    df_sport["home"].str.contains("Sport Recife", case=False, na=False),
    "Mandante",
    "Visitante"
)

df_sport["local"].value_counts()


local
Mandante     76
Visitante    76
Name: count, dtype: int64

In [14]:
ufs = pd.read_csv("../data/mandantes_vs_sport_base.csv")

# limpeza básica (pra bater os nomes)
ufs["mandante"] = ufs["mandante"].astype(str).str.strip()
ufs["mandante_nordeste"] = ufs["mandante_nordeste"].astype(str).str.strip().str.title()

df_sport["home"] = df_sport["home"].astype(str).str.strip()

# remove se já existir (evita erro de coluna duplicada)
df_sport = df_sport.drop(columns=["mandante", "mandante_nordeste"], errors="ignore")

df_sport = df_sport.merge(
    ufs[["mandante", "mandante_nordeste"]],
    left_on="home",
    right_on="mandante",
    how="left",
    validate="many_to_one"
)

# checar
df_sport[df_sport["local"] == "Visitante"]["mandante_nordeste"].value_counts(dropna=False)


mandante_nordeste
Não    62
Sim    14
Name: count, dtype: int64

In [15]:
import numpy as np

df_sport["contexto_jogo"] = np.select(
    [
        df_sport["local"] == "Mandante",
        (df_sport["local"] == "Visitante") & (df_sport["mandante_nordeste"] == "Sim"),
        (df_sport["local"] == "Visitante") & (df_sport["mandante_nordeste"] == "Não"),
    ],
    [
        "Mandante",
        "Visitante - Nordeste",
        "Visitante - Fora do Nordeste",
    ],
    default="Outro"
)

df_sport["contexto_jogo"].value_counts(dropna=False)


contexto_jogo
Mandante                        76
Visitante - Fora do Nordeste    62
Visitante - Nordeste            14
Name: count, dtype: int64

In [16]:
import numpy as np

# garantia: gols como número
df_sport["home_goal"] = pd.to_numeric(df_sport["home_goal"], errors="coerce")
df_sport["away_goal"] = pd.to_numeric(df_sport["away_goal"], errors="coerce")

df_sport["resultado"] = np.select(
    [
        # Sport mandante
        (df_sport["local"] == "Mandante") & (df_sport["home_goal"] > df_sport["away_goal"]),
        (df_sport["local"] == "Mandante") & (df_sport["home_goal"] < df_sport["away_goal"]),
        (df_sport["local"] == "Mandante") & (df_sport["home_goal"] == df_sport["away_goal"]),

        # Sport visitante
        (df_sport["local"] == "Visitante") & (df_sport["away_goal"] > df_sport["home_goal"]),
        (df_sport["local"] == "Visitante") & (df_sport["away_goal"] < df_sport["home_goal"]),
        (df_sport["local"] == "Visitante") & (df_sport["away_goal"] == df_sport["home_goal"]),
    ],
    ["Vitória", "Derrota", "Empate", "Vitória", "Derrota", "Empate"],
    default=None
)

map_pontos = {"Vitória": 3, "Empate": 1, "Derrota": 0}
df_sport["pontos"] = df_sport["resultado"].map(map_pontos)

# checagens rápidas
df_sport["resultado"].value_counts(dropna=False), df_sport["pontos"].isna().sum()


(resultado
 Vitória    68
 Empate     50
 Derrota    34
 Name: count, dtype: int64,
 np.int64(0))

In [17]:
arena = pd.read_csv("../data/sport_mandantes_arena.csv")

arena.columns


Index(['date', 'mandante_estadio'], dtype='object')

In [18]:
arena.head()


,date,mandante_estadio
0,2019-07-30,Arena de Pernambuco
1,2019-08-02,Arena de Pernambuco
2,2019-08-17,Arena de Pernambuco
3,2022-05-24,Arena de Pernambuco
4,2022-06-14,Arena de Pernambuco


In [20]:
arena = pd.read_csv("../data/sport_mandantes_arena.csv")
arena.columns, arena.head()



(Index(['date', 'mandante_estadio'], dtype='object'),
          date     mandante_estadio
 0  2019-07-30  Arena de Pernambuco
 1  2019-08-02  Arena de Pernambuco
 2  2019-08-17  Arena de Pernambuco
 3  2022-05-24  Arena de Pernambuco
 4  2022-06-14  Arena de Pernambuco)

In [21]:
# garantir datetime nos dois
arena["date"] = pd.to_datetime(arena["date"], errors="coerce")
df_sport["date"] = pd.to_datetime(df_sport["date"], errors="coerce")

# evita coluna duplicada se você já tentou antes
df_sport = df_sport.drop(columns=["mandante_estadio_arena"], errors="ignore")

df_sport = df_sport.merge(
    arena.rename(columns={"mandante_estadio": "mandante_estadio_arena"}),
    on="date",
    how="left"
)


In [22]:
df_sport["mandante_estadio"] = None

# mandante que bateu no CSV da arena -> Arena
df_sport.loc[
    (df_sport["local"] == "Mandante") & (df_sport["mandante_estadio_arena"].notna()),
    "mandante_estadio"
] = "Arena de Pernambuco"

# mandante que não bateu -> Ilha
df_sport.loc[
    (df_sport["local"] == "Mandante") & (df_sport["mandante_estadio"].isna()),
    "mandante_estadio"
] = "Ilha do Retiro"

df_sport["mandante_estadio"].value_counts(dropna=False)


mandante_estadio
None                   76
Ilha do Retiro         57
Arena de Pernambuco    19
Name: count, dtype: int64

In [23]:
df_sport[
    (df_sport["local"] == "Mandante") &
    (df_sport["mandante_estadio"] == "Arena de Pernambuco")
][["date", "away", "home_goal", "away_goal"]].sort_values("date")


,date,away,home_goal,away_goal
101,2019-07-30,Guarani SP,1.0,1.0
100,2019-08-02,Coritiba,1.0,1.0
98,2019-08-17,Botafogo SP,3.0,0.0
67,2022-05-24,CRB,0.0,1.0
64,2022-06-14,Gremio,0.0,0.0
52,2022-08-13,CSA,4.0,0.0
115,2024-04-26,Vila Nova,2.0,0.0
117,2024-05-11,Brusque,4.0,1.0
119,2024-05-18,Avai,1.0,2.0
121,2024-06-10,Paysandu,1.0,0.0


In [24]:
df_sport["contexto_final"] = np.select(
    [
        df_sport["mandante_estadio"] == "Ilha do Retiro",
        df_sport["mandante_estadio"] == "Arena de Pernambuco",
        (df_sport["local"] == "Visitante") & (df_sport["mandante_nordeste"] == "Sim"),
        (df_sport["local"] == "Visitante") & (df_sport["mandante_nordeste"] == "Não"),
    ],
    [
        "Mandante - Ilha do Retiro",
        "Mandante - Arena de Pernambuco",
        "Visitante - Nordeste",
        "Visitante - Fora do Nordeste",
    ],
    default="Outro"
)

df_sport["contexto_final"].value_counts()


contexto_final
Visitante - Fora do Nordeste      62
Mandante - Ilha do Retiro         57
Mandante - Arena de Pernambuco    19
Visitante - Nordeste              14
Name: count, dtype: int64

In [25]:
ordem = [
    "Mandante - Ilha do Retiro",
    "Mandante - Arena de Pernambuco",
    "Visitante - Nordeste",
    "Visitante - Fora do Nordeste"
]

tabela_contexto = (
    df_sport
    .groupby("contexto_final")
    .agg(
        jogos=("resultado", "size"),
        vitorias=("resultado", lambda s: (s == "Vitória").sum()),
        empates=("resultado", lambda s: (s == "Empate").sum()),
        derrotas=("resultado", lambda s: (s == "Derrota").sum()),
        pontos=("pontos", "sum"),
        gols_pro=("home_goal", lambda s: 0),  # placeholder (vamos ajustar no próximo passo)
    )
)

tabela_contexto


,jogos,vitorias,empates,derrotas,pontos,gols_pro
contexto_final,,,,,,
Mandante - Arena de Pernambuco,19,10,5,4,35,0
Mandante - Ilha do Retiro,57,39,14,4,131,0
Visitante - Fora do Nordeste,62,17,26,19,77,0
Visitante - Nordeste,14,2,5,7,11,0


In [26]:
# gols pro/contra do Sport por jogo
df_sport["gols_pro"] = np.where(df_sport["local"] == "Mandante", df_sport["home_goal"], df_sport["away_goal"])
df_sport["gols_contra"] = np.where(df_sport["local"] == "Mandante", df_sport["away_goal"], df_sport["home_goal"])
df_sport["saldo_gols"] = df_sport["gols_pro"] - df_sport["gols_contra"]

tabela_contexto = (
    df_sport
    .groupby("contexto_final")
    .agg(
        jogos=("resultado", "size"),
        vitorias=("resultado", lambda s: (s == "Vitória").sum()),
        empates=("resultado", lambda s: (s == "Empate").sum()),
        derrotas=("resultado", lambda s: (s == "Derrota").sum()),
        pontos=("pontos", "sum"),
        gols_pro=("gols_pro", "sum"),
        gols_contra=("gols_contra", "sum"),
        saldo_gols=("saldo_gols", "sum"),
    )
    .reindex(ordem)
)

# aproveitamento (%)
tabela_contexto["pontos_max"] = tabela_contexto["jogos"] * 3
tabela_contexto["aproveitamento_%"] = (tabela_contexto["pontos"] / tabela_contexto["pontos_max"] * 100).round(2)

tabela_contexto


,jogos,vitorias,empates,derrotas,pontos,gols_pro,gols_contra,saldo_gols,pontos_max,aproveitamento_%
contexto_final,,,,,,,,,,
Mandante - Ilha do Retiro,57,39,14,4,131,102.0,40.0,62.0,171,76.61
Mandante - Arena de Pernambuco,19,10,5,4,35,32.0,16.0,16.0,57,61.40
Visitante - Nordeste,14,2,5,7,11,10.0,19.0,-9.0,42,26.19
Visitante - Fora do Nordeste,62,17,26,19,77,58.0,62.0,-4.0,186,41.40


In [27]:
# Ilha vs Arena (só mandante)
ilha = tabela_contexto.loc["Mandante - Ilha do Retiro"]
arena = tabela_contexto.loc["Mandante - Arena de Pernambuco"]

comparacao_mandante = pd.DataFrame({
    "Ilha do Retiro": ilha,
    "Arena de Pernambuco": arena
})

comparacao_mandante


,Ilha do Retiro,Arena de Pernambuco
jogos,57.00,19.0
vitorias,39.00,10.0
empates,14.00,5.0
derrotas,4.00,4.0
pontos,131.00,35.0
gols_pro,102.00,32.0
gols_contra,40.00,16.0
saldo_gols,62.00,16.0
pontos_max,171.00,57.0
aproveitamento_%,76.61,61.4


In [28]:
# Visitante NE vs Fora (só visitante)
ne = tabela_contexto.loc["Visitante - Nordeste"]
fora = tabela_contexto.loc["Visitante - Fora do Nordeste"]

comparacao_visitante = pd.DataFrame({
    "Nordeste": ne,
    "Fora do Nordeste": fora
})

comparacao_visitante


,Nordeste,Fora do Nordeste
jogos,14.00,62.0
vitorias,2.00,17.0
empates,5.00,26.0
derrotas,7.00,19.0
pontos,11.00,77.0
gols_pro,10.00,58.0
gols_contra,19.00,62.0
saldo_gols,-9.00,-4.0
pontos_max,42.00,186.0
aproveitamento_%,26.19,41.4


In [29]:
tabela_contexto.to_csv("../data/tabela_aproveitamento_por_contexto.csv", index=True)
comparacao_mandante.to_csv("../data/comparacao_ilha_vs_arena.csv", index=True)
comparacao_visitante.to_csv("../data/comparacao_visitante_ne_vs_fora.csv", index=True)
